<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/code/content_analysis/bert/crime/bert/crime_classifier_BERT_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Model to classify German news articles

This is part of a project on migration attention in German news and their impact on issue attitudes. The classifier indicated whether the content is about crime.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers[torch]

In [3]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader

import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
# set random seed for reproducibility
SEED_GLOBAL = 42
np.random.seed(SEED_GLOBAL)

## Prepare data

In [5]:
# load classified migration data
print('Loading data...')
dta = pd.read_csv('https://www.dropbox.com/s/6y71ulr3axxf5iq/training_crime.csv?dl=1', sep = ',')
print('\tDone!')

Loading data...
	Done!


In [6]:
# define outcome
dta['crime_bin'] = dta.crime == 'Ja'

In [7]:
# shuffle data
dta = dta.sample(frac = 1, random_state=42).reset_index(drop = True)

In [8]:
#determine train-test-val split
splits = ['train']*1400
splits.extend(['test']*200)
splits.extend(['val']*200)
dta['split'] = splits

# Split dataset into pre-specified training, validation, and test sets
train = dta[['text', 'crime_bin']][dta['split']=='train'].reset_index(drop = True)
test = dta[['text', 'crime_bin']][dta['split']=='test'].reset_index(drop = True)
val = dta[['text', 'crime_bin']][dta['split']=='val'].reset_index(drop = True)

print(train.shape)
print(test.shape)
print(val.shape)


(1400, 2)
(200, 2)
(200, 2)


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [10]:
model_name = 'distilbert-base-german-cased'
BERT_tokenizer = AutoTokenizer.from_pretrained(model_name,
                                               model_max_length = 512)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

In [11]:
train_encodings = BERT_tokenizer(list(train['text']), padding='max_length', truncation=True, return_tensors='pt')
test_encodings = BERT_tokenizer(list(test['text']), padding='max_length', truncation=True, return_tensors='pt')
val_encodings = BERT_tokenizer(list(val['text']), padding='max_length', truncation=True, return_tensors='pt')

In [12]:
train_labels = [1 if label else 0 for label in train['crime_bin']]
test_labels = [1 if label else 0 for label in test['crime_bin']]
val_labels = [1 if label else 0 for label in val['crime_bin']]

In [13]:
import torch

class MigDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MigDataset(train_encodings, train_labels)
val_dataset = MigDataset(val_encodings, val_labels)
test_dataset = MigDataset(test_encodings, test_labels)

## Setup training, train

In [14]:
BERT_model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/270M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
BERT_model = BERT_model.to(device)
train_encodings = train_encodings.to(device)
train_labels = torch.tensor(train_labels).to(device)
val_encodings = val_encodings.to(device)
val_labels = torch.tensor(val_labels).to(device)
test_encodings = test_encodings.to(device)
test_labels = torch.tensor(test_labels).to(device)

In [16]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy="epoch",
    dataloader_pin_memory=False,
    seed=SEED_GLOBAL
)

trainer = Trainer(
    model=BERT_model,                    # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
trainer.train()

<ipython-input-13-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.300869,0.915000,0.813187,0.822222,0.804348
2,0.515900,0.239173,0.920000,0.829787,0.812500,0.847826
3,0.237800,0.263509,0.910000,0.790698,0.850000,0.739130
4,0.182200,0.268099,0.920000,0.818182,0.857143,0.782609
5,0.131000,0.401505,0.920000,0.818182,0.857143,0.782609
6,0.087500,0.418882,0.915000,0.821053,0.795918,0.847826
7,0.049100,0.560744,0.910000,0.812500,0.780000,0.847826
8,0.017000,0.566907,0.910000,0.795455,0.833333,0.760870
9,0.017000,0.654613,0.900000,0.791667,0.760000,0.826087
10,0.003100,0.653192,0.900000,0.787234,0.770833,0.804348


<ipython-input-13-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-13-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-13-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-13-bcbef893af2f>:9: UserWarning: To copy construct from a 

TrainOutput(global_step=880, training_loss=0.13907809018538417, metrics={'train_runtime': 659.9734, 'train_samples_per_second': 21.213, 'train_steps_per_second': 1.333, 'total_flos': 1854543581184000.0, 'train_loss': 0.13907809018538417, 'epoch': 10.0})

In [19]:
# save
trainer.save_model("drive/MyDrive/Bild/crime_clsfr_BERT_torch")

## Assess performance

In [20]:
trainer.evaluate(test_dataset)

<ipython-input-13-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.4659014642238617,
 'eval_accuracy': 0.93,
 'eval_f1': 0.8679245283018868,
 'eval_precision': 0.8070175438596491,
 'eval_recall': 0.9387755102040817,
 'eval_runtime': 2.827,
 'eval_samples_per_second': 70.746,
 'eval_steps_per_second': 1.415,
 'epoch': 10.0}